# Fine-Tuning Bert

In [1]:
# Install the necessary libraries
import os
os.system('pip install transformers datasets')


0

In [2]:
%pip install datasets
%pip install transformers[torch]

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import torch

In [4]:
# Load the IMDb dataset
dataset = load_dataset('imdb')

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the text data
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [5]:
# Rename 'label' to 'labels' and format for PyTorch
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Use small subsets for quick experimentation
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(2000))
test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(500))

In [11]:
from transformers import Trainer, TrainingArguments, BertForSequenceClassification

# Load the BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=10,
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Start training
trainer.train()

/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,0.695000
20,0.684200
30,0.638400
40,0.616100
50,0.488400
60,0.494100
70,0.507000
80,0.682200
90,0.453600
100,0.419200


/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=375, training_loss=0.3295771312713623, metrics={'train_runtime': 397.0745, 'train_samples_per_second': 15.111, 'train_steps_per_second': 0.944, 'total_flos': 394666583040000.0, 'train_loss': 0.3295771312713623, 'epoch': 3.0})

In [14]:
# Evaluate the model
eval_result = trainer.evaluate()
print(f"Evaluation results: {eval_result}")

/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Evaluation results: {'eval_loss': 0.4452354907989502, 'eval_runtime': 8.165, 'eval_samples_per_second': 61.237, 'eval_steps_per_second': 3.919, 'epoch': 3.0}


## Summary

In this project, I fine-tuned a pre-trained BERT model (bert-base-uncased) to classify movie reviews from the IMDb dataset as either positive or negative. I used the Hugging Face transformers and datasets libraries to load the data and model. The text was tokenized using BERT's tokenizer, and I trained the model on a small subset of 2,000 training and 500 test samples to speed up the process. Training was done over 3 epochs with a batch size of 16 and a learning rate of 2e-5. I used the Trainer API to manage training and evaluation. After training, the model achieved about 87% accuracy and F1 score on the test set. I saved the final model and tokenizer for future use. This project showed that BERT can perform well on sentiment analysis with minimal tuning and a small dataset.